En las palabras de Philip Wadler...

<video controls src="lambda.mp4" />

Clip anterior: Extraído de ["Propositions as Types"](https://youtu.be/IOiZatlZtGU) por Philip Wadler, en Strange Loop Conference 2015.

# Isomorfismo Curry - Howard

- Existe una correspondencia directa entre proposiciones lógicas y el cálculo lambda.

- El sistema de verificación de tipos de Haskell funciona como un verificador de demostraciones.

## Construcción del sistema

Necesitamos una forma de representar elementos fundamentales de la lógica proposicional, en particular los valores de verdad o falsedad y operadores básicos.

In [1]:
-- Valores de verdad
data Verdadero = Verdadero
data Falso

In [2]:
-- Operadores
data Y a b = Y a b
data O a b = OIzquierdo a | ODerecho b

¿Qué onda con la negación?

- Lógica Intuicionista (o constructivista)

$$ \lnot A \equiv A \implies\bot $$

In [3]:
type No a = a -> Falso

- Variables Proposicionales $\Leftrightarrow$ Variables de tipos
- Implicación $\Leftrightarrow$ Tipo función `a -> b`.

## Cuantificación

- En Haskell, todas las firmas de funciones con variables de tipos están cuantificadas universalmente.
- Si algo se cumple para cualquier tipo, es equivalente decir que la proposición se cumple para cualquier variable.

In [4]:
pi :: Double
pi = 3.14159

intercambio :: (a, b) -> (b, a)
intercambio (x, y) = (y, x)

## Demostraciones constructivistas

Si tenemos lo siguiente:

$$ \dfrac{A \implies B \\ A}{\therefore B} $$

podemos verlo de la siguiente manera:

$$ (A \implies B) \implies A \implies B $$

Evidenciar una función que cumpla con un cierto tipado, es equivalente a demostrar que se cumple la proposición isomorfa.

```haskell
modusPonens :: (a -> b) -> a -> b
```

es equivalente a demostrar que de $a\implies b$ y $a$ se sigue $b$.

## Ejemplos

Demuestra:

$$ \dfrac{A}{\therefore A} $$

In [5]:
identidad :: a -> a
identidad a = a

In [11]:
:t id

id :: forall a. a -> a

Demuestra:

$$ \dfrac{A\\ B}{\therefore A} $$

In [6]:
ignora :: a -> b -> a
ignora a _ = a

In [12]:
:t const

const :: forall a b. a -> b -> a

Demuestra Modus Ponens:

$$ \dfrac{A\implies B\\A}{\therefore B} $$

In [8]:
modusPonens :: (a -> b) -> a -> b
modusPonens f x = f x

Line 2: Eta reduce
Found:
modusPonens f x = f x
Why not:
modusPonens f = f

In [10]:
:t ($)

($) :: forall a b. (a -> b) -> a -> b

Demuestra el Silogismo Hipotético:

$$ \dfrac{B \implies C\\ A\implies B}{\therefore A\implies C} $$

In [13]:
silogismoHipotetico :: (b -> c) -> (a -> b) -> (a -> c)
silogismoHipotetico = (.)

Demuestra la contrapositiva:

$$ \dfrac{A \implies B\\\lnot B}{\therefore \lnot A} $$

In [14]:
-- contrapositiva :: (a -> b) -> (b -> Falso) -> (a -> Falso)
contrapositiva :: (a -> b) -> No b -> No a
contrapositiva f noB = noB . f

Demuestra la primera ley de DeMorgan:

$$ \dfrac{\lnot A \land \lnot B}{\therefore \lnot (A\lor B)} $$

In [19]:
deMorgan1 :: Y (No a) (No b) -> No (O a b)
deMorgan1 (Y noA noB) = \o -> case o of
                                OIzquierdo a -> noA a
                                ODerecho   b -> noB b

Line 2: Redundant lambda
Found:
deMorgan1 (Y noA noB)
  = \ o
      -> case o of
           OIzquierdo a -> noA a
           ODerecho b -> noB b
Why not:
deMorgan1 (Y noA noB) o
  = case o of
      OIzquierdo a -> noA a
      ODerecho b -> noB bLine 2: Use lambda-case
Found:
\ o
  -> case o of
       OIzquierdo a -> noA a
       ODerecho b -> noB b
Why not:
\case
  OIzquierdo a -> noA a
  ODerecho b -> noB b

Demuestra la segunda ley de DeMorgan:

$$ \dfrac{\lnot A\lor\lnot B}{\therefore \lnot (A\land B)} $$

In [21]:
--                              (Y a b) -> Falso
deMorgan2 :: O (No a) (No b) -> No (Y a b)
deMorgan2 (OIzquierdo noA) = \(Y a b) -> noA a
deMorgan2 (ODerecho   noB) = \(Y a b) -> noB b

Demuestra la introducción de la doble negación:

$$ \dfrac{A}{\therefore \lnot\lnot A} $$

Demuestra la conmutatividad de la conjunción:

$$ \dfrac{A\land B}{\therefore B\land A} $$

Demuestra que la conjunción implica la disjunción:

$$ \dfrac{A\land B}{\therefore A \lor B} $$

Demuestra la eliminación de la conjunción:

$$ \dfrac{A \land B}{\therefore A} $$

Demuestra:

$$ \dfrac{A\implies B\implies C}{\therefore B\implies A\implies C} $$

Demuestra la dominancia de la disyunción:

$$ \dfrac{}{\therefore \top\lor A} $$

Demuestra la identidad de la conjunción:

$$ \dfrac{\top\land A}{\therefore A} $$

Demuestra la introducción de la disyunción:

$$ \dfrac{A}{\therefore A \lor B} $$

Demuestra la idempotencia:

$$ \dfrac{A \lor A}{A} $$

Demuestra la conmutatividad de la disyunción:

$$ \dfrac{A\lor B}{B\lor A} $$

Demuestra:

$$ \dfrac{A\land B}{B\lor A} $$

# Recursos

- ["Propositions as Types" by Philip Wadler. Strange Loop Conference](https://youtu.be/IOiZatlZtGU)
- [Stephen Pimentel - Propositions as Types for Beginners in Haskell - λC 2018](https://youtu.be/tfG7T54MhIU)